# Feature Calculation

__UbiComp Assignment 02, Task 01:__
This second notebook allows you to calculate selected features and corresponding labels that are needed for the classification of gaze data.

Parts of this code are taken from a tutorial from Pupil Labs. See Section 3 in their [repository](https://github.com/pupil-labs/pupil-tutorials/blob/master/01_load_exported_data_and_visualize_pupillometry.ipynb).


## The Dispersion-Threshold Identification I-DT algorithm

We start with defining some threshold parameters needed for detecting the fixations, as described in [Salvucci and Goldberg 2000](https://dl.acm.org/doi/abs/10.1145/355017.355028)

In [ ]:
import numpy as np

# spatial threshold or the dispersion itself
max_dispersion = np.deg2rad(1.6)
# temporal threshold or duration
min_duration = 100

## 1. Some auxilary functions for the fixation calculation from raw gaze data

Now we need several functions that help us detecting the fixations.

### 1.1 Vector Dispersion
This function (from the Pupil Labs Tutorial) calculates the dispersion of a given list of (gaze) vector.

In [ ]:
import scipy.spatial.distance

def vector_dispersion(vectors):
    distances = scipy.spatial.distance.pdist(vectors, metric='cosine')
    distances.sort()
    cut_off = np.max([distances.shape[0] // 5, 4])
    return np.arccos(1. - distances[-cut_off:].mean())

### 1.2 Dispersion Calculation
We adapted the following function (also from the Pupil Labs Tutorial) so that it works with the Microsft HoloLens2 gaze data. 

In [ ]:
def gaze_dispersion(eye_data):
    base_data = eye_data

    vectors = []
    for p in base_data:
        vectors.append((p['gazeDirection_x'], p['gazeDirection_y'], p['gazeDirection_z']))
    vectors = np.array(vectors, dtype=np.float32)

    if len(vectors) < 2:
        return float("inf")
    else:
        return vector_dispersion(vectors)  


### 1.3 Centroid Calculation
The following function calculates the centroid of each group of points that is identified as a fixation.

In [ ]:
def get_centroid(eye_data):
    '''Calculates the centroid for each point in a df of points.
    Input: Df of points.
    Output: Vector containg the centroid of all points.'''
    x = [p['gazeDirection_x'] for p in eye_data]
    y = [p['gazeDirection_y'] for p in eye_data]
    z = [p['gazeDirection_z'] for p in eye_data]
    return (sum(x) / len(eye_data), sum(y) / len(eye_data), sum(z) / len(eye_data))

### 1.4 Fixation Detection
The following function calculates the fixations (also adapted from the Pupil Labs Tutorial).

We changed the column names, added centroid calculation, and changed the output format to the dictionary.

In [ ]:
from collections import deque

def detect_fixations(gaze_data):
    # Convert Pandas data frame to list of Python dictionaries
    gaze_data = gaze_data.T.to_dict().values()

    candidate = deque()
    future_data = deque(gaze_data)
    while future_data:
        # check if candidate contains enough data
        if len(candidate) < 2 or candidate[-1]['eyeDataTimestamp'] - candidate[0]['eyeDataTimestamp'] < min_duration:
            datum = future_data.popleft()
            candidate.append(datum)
            continue

        # Minimal duration reached, check for fixation
        dispersion = gaze_dispersion(candidate)
        if dispersion > max_dispersion:
            # not a fixation, move forward
            candidate.popleft()
            continue

        # Minimal fixation found. Try to extend!
        while future_data:
            datum = future_data[0]
            candidate.append(datum)

            dispersion = gaze_dispersion(candidate)
            if dispersion > max_dispersion:
                # end of fixation found
                candidate.pop()
                break
            else:
                # still a fixation, continue extending
                future_data.popleft()
        centroid = get_centroid(candidate)
        yield {"start": candidate[0]['eyeDataTimestamp'], "end": candidate[-1]['eyeDataTimestamp'],
               "duration": candidate[-1]['eyeDataTimestamp'] - candidate[0]['eyeDataTimestamp'],
              "centroid": centroid, "dispersion": dispersion}
        candidate.clear()


## 2. Calculation of Gaze Features



### 2.1 Valid Data
Input: Raw gaze data from the Microsoft Hololens2 (through the framework [ARETT](https://github.com/AR-Eye-Tracking-Toolkit/ARETT)).\
Output: Only valid gaze points 

In [ ]:
import pandas as pd

def only_valid_data(data):
    '''Returns only valid gaze points. Those have values in gazeDirection_x etc.'''
    return data[(data.gazeHasValue == True) & (data.isCalibrationValid == True)]

### 2.2 Blink calculation

Whenever `gazeHasValue == False` we assume it's a blink. Consequetive rows without data (i.e.  `gazeHasValue == False`) are counted as one single blink.

We calculate __five features__ based on the blinks: 
- number of blinks
- mean duration of blinks
- maximum duration of blinks
- minimum duration of blinks
- blink rate


In [ ]:
import datetime
def calculate_blink_features(df, timespan): 
    ''' Calculates the blink features for a given df of raw data.
    Input: Dataframe with raw data (incl. invalid points), timespan of data chunk in seconds\
    Output: Dict with the blink features
    '''
    i=0;
    blink_list = []
    blink_duration_list = []
    number_of_blinks = 0
    window_start_time = df["eyeDataTimestamp"][0]
    window_end_time = df["eyeDataTimestamp"][0]
    all_false = 0;
    if (not window_start_time  or not window_end_time):
        return {}
        
    for i, row in df.iterrows():    
        
        cur_number_of_blinks = 0
        if (not row["gazeHasValue"]):
            cur_number_of_blinks+=1
            all_false += 1
            while (i < len(df["gazeHasValue"])) and (not df["gazeHasValue"][i]) and  window_end_time - window_start_time < 1000:
                window_end_time = row["eyeDataTimestamp"]
                i+=1
                all_false +=1
            blink_list.append(cur_number_of_blinks)
            duration = window_end_time - window_start_time
            blink_duration_list.append(duration)
        
        number_of_blinks += cur_number_of_blinks;
        if  window_end_time - window_start_time > 1000:
            window_start_time = window_end_time
    
    blinks_per_second = 0
    if (len(blink_list)  > 0):
        blinks_per_second = number_of_blinks / timespan 
    avg_blink_duration = 0
    min_blink_duration = 0
    max_blink_duration = 0
    if (len(blink_duration_list) > 0):
        avg_blink_duration = sum(blink_duration_list) / len(blink_duration_list)
        min_blink_duration = min(blink_duration_list)
        max_blink_duration = max(blink_duration_list)
        
    # print("all_blinks: ", number_of_blinks, " avg_blink_duration: ", avg_blink_duration, 
    #    " min_blink_duration: ", min_blink_duration, " max_blink_duration: ", max_blink_duration,
    #  " blinks_per_second: ", blinks_per_second, " all false: ", all_false)
    
    return {"number_of_blinks":number_of_blinks, "blinkMean": avg_blink_duration, 
            "blinkMin": min_blink_duration, "blinkMax": max_blink_duration, 
            "blinkRate": blinks_per_second}


### 2.3 Calculate Fixation Features
The following function calculates __11 features__ based on the fixations. 
We need the following features:
- minimal fixation duration
- maximal fixation duration
- mean fixation duration
- variance of the fixation duration
- standard deviation of the fixation duration
- minimal fixation dispersion
- maximal fixation dispersion
- mean fixation dispersion
- variance of the fixation dispersion
- standard deviation of the fixation dispersion
- fixation frequency per second

In [ ]:
def calculate_fixation_features(df_fixations, timespan):
    '''Calculates the fixation features. 
    Input: Dataframe with fixation, timespan of data chunk in seconds.
    Output: Dict containing the fixation features.'''

    min_fix = df_fixations["duration"].min()
    max_fix = df_fixations["duration"].max()
    mean_fix = df_fixations["duration"].mean()
    var_fix = df_fixations["duration"].var()
    std_fix = df_fixations["duration"].std()
    
    min_dispersion = df_fixations["dispersion"].min()
    max_dispersion = df_fixations["dispersion"].max()
    mean_dispersion = df_fixations["dispersion"].mean()
    var_dispersion = df_fixations["dispersion"].var()
    std_dispersion = df_fixations["dispersion"].std()
    
    fixation_frequency_second = (len(df_fixations["dispersion"]) / timespan)
    
    # print("min: ", min_fix, " max: ", max_fix, " mean: ", mean_fix, " var: ", var_fix, "std: ", std_fix)
    # print("min dispersion: ", min_dispersion, " max: ", max_dispersion, " mean: ", mean_dispersion, 
    #      " var: ", var_dispersion)
    #print("x dispersion: ", dispersion_x, " y dispersion: ", dispersion_y, " z dispersion: ", dispersion_z)
    
    return {"meanFix": mean_fix, "minFix": min_fix, "maxFix": max_fix, "varFix": var_fix, "stdFix": std_fix,
            "meanDis": mean_dispersion, "minDis": min_dispersion, "maxDis": max_dispersion,
            "varDis": var_dispersion, "stdDisp": std_dispersion,
            "freqDisPerSec": fixation_frequency_second}
    

def get_fixation_df(df_valid):
    '''Calls function to calculate Fixations. Converts the list of fixations to a dataframe and numbers the rows as index.
     Input: Dataframe containg valid gaze points.
     Output: Dataframe containing the fixation features.'''
    fixations = list(detect_fixations(df_valid))
    df = pd.DataFrame(fixations)
    df['index'] = range(1, len(df) + 1)
    return df

### 2.4 Direction of the Succesive Fixation Points

This function calculates the dominant direction along the __x__ and __y__ axis (i.e., __two addtional features__).

First for each x- and y-value it is calulated whether it is larger then the point in the list before (the points are ordered by time).
Then all the True-values in this list are summed up and divided by the length of the list.

As a result, e.g., if (xDir == 1), then each x-value is further right than its predecessors in the list.

If xDir > 0.5 the x-direction is (mostly) to the right. \
If xDir = 0.5 the x-direction is equally to both sides. \
If xDir < 0.5 the x-direction is (mostly) to the left. \
If yDir > 0.5 the y-direction is (mostly) to the top. \
If yDir = 0.5 the y-direction is equally to both sides. \
If yDir < 0.5 the y-direction is (mostly) to the bottom.

In [ ]:
def calculate_directions_of_list(points):
    '''Calculates the dominant direction of points.
    Input: Dataframe containing fixation points.
    Output: Dict with dominant direction for x (xDir) and y (yDir).
    '''
    x_values, y_values, z_values = zip(*points['centroid'])
    # Get a list of whether a given value is greater then the previous one in the list
    res_x = [float(val1) < float(val2) for val1, val2 in zip(x_values, x_values[1:])]
    # Sum all that are True
    sum_x = sum(res_x)
    # Divide the sum by the total number of values to get the desired output.
    # dir_x is -1 if there are no fixation (i.e. prevent division by zero)
    dir_x = -1
    if len(res_x) != 0:
        dir_x = sum_x/len(res_x)
        
    res_y = [float(val1) < float(val2) for val1, val2 in zip(y_values, y_values[1:])]
    sum_y = sum(res_y)
    dir_y = -1
    if len(res_y) != 0:
        dir_y = sum_y /len(res_y)
      
    
    return {"xDir": dir_x, "yDir": dir_y}

### 2.5 Fixation Density
The number of fixations per area (i.e., the area from where all gaze points are collected). 


In [ ]:
def calculate_fixation_density(df_all, df_fix):
    '''Calculates the fixation density per area.
    Input: Dataframe with all valid gazepoints, Dataframe with fixations.
    Output: Dict containing the fixation density.'''
    min_x = df_all['gazeDirection_x'].min()
    min_y = df_all['gazeDirection_y'].min()
    max_x = df_all['gazeDirection_x'].max()
    max_y = df_all['gazeDirection_y'].max()
    
    length = abs(max_x-min_x)
    height = abs(max_x-min_x)
    area = length*height
    
    number_of_fixations = len(df_fix)
    
    fix_dens = -1
    if area != 0: 
        fix_dens = number_of_fixations/area
    return {"fixDensPerBB": fix_dens}
    

### 2.6 Get Features for a given *timespan* (in seconds)
This function makes use of the feature calculations explained above.\
It splits the data into chunks based on a given timespan and discards the final chunk if its duration is shorter than the *timespan*.\
Finally, it returns the feature-chunks. 

In [ ]:
import tabulate
from sklearn import preprocessing

def get_features_for_n_seconds(df, timespan, label, participant_id):
    ''' Calculates the features for a raw gaze data in chunks of n seconds.
    Input: Dataframe with raw gaze data, timespan to chunk, label (i.e. activity class).
    Output: List of dictionaries, one dictionary contains a chunk of features.
    '''
    
    list_of_features = []
    i = 0
    while i < len(df)-1: 
        newdf = pd.DataFrame(columns=df.columns)
        start_time = df["eyeDataTimestamp"][i]  
        
        while i < len(df)-1 and df["eyeDataTimestamp"][i] < (start_time+timespan*1000):
            entry = df.iloc[[i]]
            newdf = pd.concat([newdf,entry])
            i+=1
        newdf.reset_index(inplace = True)
        
        if (len(newdf) > timespan*28):
            newdf_valid = only_valid_data(newdf)
            df_fixations = get_fixation_df(newdf_valid)
            
            features = calculate_fixation_features(df_fixations, timespan)
            blinks = calculate_blink_features(newdf,timespan)

            directions = calculate_directions_of_list(df_fixations)            
            density = calculate_fixation_density(newdf_valid, df_fixations)
            
            features.update(blinks)
            features.update(directions)
            features.update(density)
            features["label"] = label
            # print(f"label: {label}")
            features["duration"] = timespan
            features["participant_id"] = participant_id            
            list_of_features.append(features)   
        
    return list_of_features


## 3 Assemble the Features and Write them to a CSV File

### 3.1 Save the features in a CSV file

In offline processing (i.e., with the previously collected gaze data) we write to the csv file (*w*).\
In online processing (i.e., gaze data collected live from an eye tracker), we append to the csv file (*a*).

In [ ]:
import tabulate
import csv
def save_as_csv(list_of_dict, participant, folder):
    '''Saves a list of dicts as one csv file.
    Input: List of feature dicts, participant id.
    Output: CSV file, saved in same directory as this file.'''
    header = list_of_dict[0].keys()
    rows =  [x.values() for x in list_of_dict]

    keys = list_of_dict[0].keys()
    file_name = os.path.join(folder, f'feature_list_P{participant}.csv')
    # if the file already exists, append the rows
    if os.path.exists(file_name):
        with open(file_name, 'a', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writerows(list_of_dict)
    # otherwise create a new file
    else:
        with open(file_name, 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(list_of_dict)
    # '''        
    file_name_all = os.path.join(folder, f'feature_list_all.csv')
    # if the file already exists, append the rows
    if os.path.exists(file_name_all):
        with open(file_name_all, 'a', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writerows(list_of_dict)
    # otherwise create a new file
    else:
        with open(file_name_all, 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(list_of_dict)
    # '''
    return file_name

### 3.2 Collect the data from csv files
This function reads gaze data from all csv files in one folder. 
Based on the filename, it extracts the activity and the participant ID. 

You need to put the data in a folder that is defined by the variable `root_dir` or change the path accordingly.
The filenames need to be in this form: *00_reading.csv*, where the number denotes the particpant ID and the second part the activity.

The output dict has this form:
```
{ "01" :
    { "reading" : "./Data/RawGazeData/00_reading.csv",
      "search" : ...
    },
    ...
```

In [ ]:
import os
def collect_data_from_csv_files():
    ''' Collects the filenames of all csv-files in a given folder.
    Input: -
    Output: Dict containing the activites per participants.'''
    root_dir = "./Data/RawGazeData/"
    
    files_list = os.listdir(root_dir)
    print(files_list)
    # filenames are like this: 00_reading.csv, 01_reading.csv,...
    # where the "00" etc. indicates the participant number
    df_files = {}
    for index, path in enumerate(files_list):
        if ("csv" in path):
            name = path.split("_")
            participant_id = name[0]
            activity = name[1].split(".")[0]
            if df_files.get(participant_id):
                df_files[participant_id][activity] = f"{root_dir}{path}"
            else:
                df_files.update({participant_id: {activity: f"{root_dir}{path}"}})
    return df_files

### 3.3 Prepare the Features for Classification
It starts the calculation of features by calling the functions from above.

In [ ]:
def calculate_features_and_save_for_list_of_files():
    '''Calculates the features for multiple gaze data files and saves those as CSV files.
    Input: Dict containg the activity class (key) and the paths to the gaze data files (value)
    Output: Feature-files.'''
    
    paths = collect_data_from_csv_files()
    for participant_id, participant_item in paths.items():
        feature_list = []
        for activity, path in participant_item.items():
            print(f"calculating features for : {path}")
            df = pd.read_csv(path)
            print(f"activity: {activity}")
            feature_list.append(get_features_for_n_seconds(df, 20, activity, participant_id))

        flat_ls = [item for sublist in feature_list for item in sublist]
        # change the folder here to not overwrite the data we provided!
        save_as_csv(flat_ls, participant_id, './Data/FeatureFiles/')
        
    print("done.")  

# Uncomment this line to calculate the features from the raw gaze data.
# calculate_features_and_save_for_list_of_files()